In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)

/home/kanishev/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/kanishev/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/kanishev/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/kanishev/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.

Epoch 1/5
60000/60000 [==============================] - 2s 32us/sample - loss: 0.2986 - acc: 0.9125
Epoch 2/5
60000/60000 [==============================] - 2s 31us/sample - loss: 0.1436 - acc: 0.9574
Epoch 3/5
60000/60000 [==============================] - 2s 31us/sample - loss: 0.1093 - acc: 0.9665
Epoch 4/5
60000/60000 [==============================] - 2s 33us/sample - loss: 0.0894 - acc: 0.9725
Epoch 5/5
10000/10000 [==============================] - 0s 18us/sample - loss: 0.0753 - acc: 0.9765


[0.0753172178490553, 0.9765]

In [3]:
_ , nrows , ncols = model.input_shape

def mkbod(lst):
    b = "\n".join(map(lambda x: "  ; " + x, lst))
    b = "  {" + b[3:] + "\n  }"
    return b

code = "type row = \n"
code += mkbod(["p{:02} : real".format(c) for c in range(ncols)])
code += ";;\n\n"

code += "type image = \n"
code += mkbod(["r{:02} : row".format(c) for c in range(nrows)])
code += ";;\n\n"

code += "type output = \n"
code += mkbod(["out{} : real".format(i) for i in range(10)])
code += ";;\n\n"

typedecls = code

import itertools
input_flat_vector = \
    [ "x.r{:02}.p{:02}".format(r,c) 
     for r,c in itertools.product(range(nrows), range(ncols)) ]

In [4]:
print(typedecls)

type row = 
  { p00 : real
  ; p01 : real
  ; p02 : real
  ; p03 : real
  ; p04 : real
  ; p05 : real
  ; p06 : real
  ; p07 : real
  ; p08 : real
  ; p09 : real
  ; p10 : real
  ; p11 : real
  ; p12 : real
  ; p13 : real
  ; p14 : real
  ; p15 : real
  ; p16 : real
  ; p17 : real
  ; p18 : real
  ; p19 : real
  ; p20 : real
  ; p21 : real
  ; p22 : real
  ; p23 : real
  ; p24 : real
  ; p25 : real
  ; p26 : real
  ; p27 : real
  };;

type image = 
  { r00 : row
  ; r01 : row
  ; r02 : row
  ; r03 : row
  ; r04 : row
  ; r05 : row
  ; r06 : row
  ; r07 : row
  ; r08 : row
  ; r09 : row
  ; r10 : row
  ; r11 : row
  ; r12 : row
  ; r13 : row
  ; r14 : row
  ; r15 : row
  ; r16 : row
  ; r17 : row
  ; r18 : row
  ; r19 : row
  ; r20 : row
  ; r21 : row
  ; r22 : row
  ; r23 : row
  ; r24 : row
  ; r25 : row
  ; r26 : row
  ; r27 : row
  };;

type output = 
  { out0 : real
  ; out1 : real
  ; out2 : real
  ; out3 : real
  ; out4 : real
  ; out5 : real
  ; out6 : real
  ; out7 : real
  ; o

In [8]:
# obtain weights - generate iml
weights = model.get_weights()

code = ["let nn x = "]

# First layer
for j in range(weights[0].shape[1]):
    letin = []
    for i in range(weights[0].shape[0]):
        letin.append("({:.3f}) *. {}".format(weights[0][i][j],input_flat_vector[i]))
    letin.append("({:.3f})".format(weights[1][j]))
    letin = "  let a{:03} = relu @@ ".format(j) + " +. ".join(letin) + "  in"
    code.append(letin)

# Second layer
for j in range(weights[2].shape[1]):
    letin = []
    for i in range(weights[2].shape[0]):
        letin.append("({:.3f}) *. {}".format(weights[2][i][j],"a{:03}".format(i)))
    letin.append("({:.3f})".format(weights[3][j]))
    letin = "  let out{} = ".format(j) + " +. ".join(letin) + "  in"
    code.append(letin)
    
# Output
code.append("  { " + ";".join(["out{}".format(i) for i in range(10)]) + " }")


In [9]:
with open("nn.iml","w") as f:
    f.write("let relu x = Real.(if x > 0.0 then x else 0.0);;\n\n")
    
    f.write(typedecls)
    f.write("\n\n")
    f.write("\n".join(code))
    f.write(";;\n\n")

In [10]:
## Sample images

In [17]:
N = 10
images = x_test[np.random.randint(x_test.shape[0],size=N)]
flatten_model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28))
])
images = flatten_model.predict(images)

code = []
for i in range(N):
    images[i]

In [57]:
code = []
for i in np.random.randint(x_test.shape[0],size=N):
    imcode = []
    for j in range(x_test.shape[1]):
        row = ["p{:02}=({:.3f})".format(n,p) for n,p in enumerate(x_test[i][j])]
        row = "  ; r{:02}={{".format(j) + ";".join(row) + "}"
        imcode.append(row)
    imcode = "\n".join(imcode)
    imcode = "  {" + imcode[3:] + "\n  }\n;;\n\n"
    imcode = "let image_{} = \n".format(i) + imcode 
    code.append(imcode)

In [59]:
with open("samples.iml", "w") as f:
    f.write("\n".join(code))

In [62]:
28 * 28

784